In [ ]:
## IMPORTS

import os 
import time
from io import StringIO
import numpy as np
import pandas as pd

import openpyxl

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from numpy.polynomial.polynomial import Polynomial
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error


import jou_gen as jg

import ansys.fluent.core as pyfluent

# Initialisation

In [ ]:
## Initialisation

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI'
# File > Application > Start > folder_path
# Ensuite copier le numéro du server et le modifier ci-dessous
solver_path = folder_path + '\\' + 'server_info-209712.txt'
# Créer une session
solver = pyfluent.connect_to_fluent(server_info_filepath=solver_path)
tui = solver.tui

In [134]:
## Méthodes

fp_cmd = "D:\\ANSYS Fluent Projects\\temp"

def create_named_expression(tui, named_expression, definition, unit) :
    string_list = [f"""define/named-expressions/add \"{named_expression}\" definition "{definition} [{unit}]" quit"""]
    concatenate_and_write_to_file(string_list,fp_cmd+'\\create_named_expression.txt')
    tui.file.read_journal(f'"{fp_cmd}\\create_named_expression.txt"')

def create_field(tui, named_expression, definition) :
    string_list = [f"""define/named-expressions/add \"{named_expression}\" definition "{definition}" quit"""]
    concatenate_and_write_to_file(string_list,fp_cmd+'\\create_field.txt')
    tui.file.read_journal(f'"{fp_cmd}\\create_field.txt"')

def change_gravity(tui, theta): # theta en degrés
    theta_rad = np.deg2rad(theta)
    gravity_y = 9.81*np.sin(theta_rad)
    gravity_z = 9.81*np.cos(theta_rad)
    string_list = [f"""define/operating-conditions/gravity yes 0 {gravity_y} {gravity_z} quit"""]
    concatenate_and_write_to_file(string_list,fp_cmd+'\\change_gravity.txt')
    tui.file.read_journal(f'"{fp_cmd}\\change_gravity.txt"')

In [135]:
## Initialisation des Inputs 
# On initialise les inputs avec l'excel pour créer toutes les variables et ensuite Inputs sera modifié via PVT-thermal-performance-model

Inputs = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')
# Input un tableau de 4 colonnes : named_expression, value, unit, comment

nb_hx = int(Inputs[Inputs['named_expression'] == 'nb_hx'].iloc[0]['value'])

In [ ]:
## Importation des données sur PyFluent

for i in range(len(Inputs)):
    named_expression = Inputs.iloc[i]['named_expression']
    value = Inputs.iloc[i]['value']
    unit = Inputs.iloc[i]['unit']
    create_named_expression(tui, named_expression, value, unit)
    print(str(100*i/len(Inputs)) + "% des expressions créées")

print('Inputs importés')

In [136]:
## Modification de la gravité

# On modifie les composantes de la gravité en fonction de l'inclinaison du panneau
theta = Inputs.loc[Inputs['named_expression'] == 'theta']['value'].values[0]
change_gravity(tui, theta)

In [ ]:
## Initialisation des champs des températures et des températures moyennes

# Créé un champ de température pour toutes les parties de l'échangeur
y0, z0 = 0, 0
for i in range(1, nb_hx+1) :
    theta_i = Inputs.loc[Inputs['named_expression'] == f'theta_{i}']['value'].values[0]
    theta_i = np.deg2rad(theta_i)
    L_i = Inputs.loc[Inputs['named_expression'] == f'L_{j}']['value'].values[0]
    create_field(tui, f'T_field_{i}', f"(T_fluid_in_{i} + (b_f_{i} / a_f_{i})) * exp(a_f_{i} * -{np.sin(theta_i)}*(Position.y-{y0}) + {np.cos(theta_i)}*(Position.z-{z0})) - (b_f_{i} / a_f_{i})")
    create_field(tui, f'T_mean_{i}', f'(T_fluid_in_{i} + (b_f_{i} / a_f_{i}))/(a_f_{i}*L_{i}) * exp(a_f_{i} * L_{i}) - (T_fluid_in_{i} + (b_f_{i} / a_f_{i}))/(a_f_{i}*L_{i})- (b_f_{i} / a_f_{i})')
    y0 = y0 - L_i*np.sin(theta_i)
    z0 = z0 + L_i*np.cos(theta_i)
    print(str(100*i/(nb_hx+1)) + "% des champs créées")

print('Initialisation terminée')

In [137]:
## Température moyenne pour la partie centrale 

Inputs.loc[len(Inputs)+1] = ['T_mean_345', 0, 'K', 'Température moyenne de la partie centrale'] 

In [ ]:
## Initialisation des boundary conditions sur ANSYS

# NB on pourrait les initialiser depuis Pyfluent mais ce serait se casser la tête pour rien

Fin de l'initialisation

# Début de la boucle

In [ ]:
## Appel du PVT-thermal-performances-model

# On initialise la première température avec le modèle et le transfert radiatif
# Inputs[Inputs['named_expression']=='T_fluid_in_0'].iloc[0]['value'] = ht.
# Inputs[Inputs['named_expression']=='Qdot_back_rad'].iloc[0]['value'] = ht.Qdot_back_rad

# On initialise les a_f et b_f avec le modèle
# for i in range(1, nb_hx+1):
#     Inputs[Inputs['named_expression'] == f'a_f_{i}'].iloc[0]['value'] = ht.
#     Inputs[Inputs['named_expression'] == f'b_f_{i}'].iloc[0]['value'] = ht.

In [122]:
## Fonctions pour calculer les températures

def T_fluid_out(T_fluid_in, L, a_f, b_f):
    T_fluid_out = (T_fluid_in + (b_f / a_f)) * np.exp(a_f * L) - (b_f / a_f)
    return T_fluid_out

def T_fluid_mean(T_fluid_in, L, a_f, b_f):
    T_fluid_mean = (T_fluid_in + (b_f / a_f))/(a_f*L) * np.exp(a_f * L) - (T_fluid_in + (b_f / a_f))/(a_f*L)- (b_f / a_f)
    return T_fluid_mean

In [ ]:
## Création des champs de températures

for i in range(1,nb_hx+1) :
    T_fluid_in = Inputs[Inputs['named_expression'] == f'T_fluid_in_{i}'].iloc[0]['value']
    L = Inputs[Inputs['named_expression'] == f'L_{i}'].iloc[0]['value']
    a_f = Inputs[Inputs['named_expression'] == f'a_f_{i}'].iloc[0]['value']
    b_f = Inputs[Inputs['named_expression'] == f'b_f_{i}'].iloc[0]['value']
    Inputs[Inputs['named_expression'] == f'T_fluid_in_{i+1}'] = T_fluid_out(T_fluid_in, L, a_f, b_f)


# Température moyenne pour la partie centrale
T_fluid_in = Inputs[Inputs['named_expression'] == 'T_fluid_in_3'].iloc[0]['value']
L_3 = Inputs[Inputs['named_expression'] == 'L_3'].iloc[0]['value']
L_4 = Inputs[Inputs['named_expression'] == 'L_4'].iloc[0]['value']
L_5 = Inputs[Inputs['named_expression'] == 'L_5'].iloc[0]['value']
a_f_3 = Inputs[Inputs['named_expression'] == 'a_f_5'].iloc[0]['value']
b_f_3 = Inputs[Inputs['named_expression'] == 'b_f_5'].iloc[0]['value']

Inputs[Inputs['named_expression'] == f'T_mean_345'].iloc[0]['value'] = T_fluid_mean(T_fluid_in, L_3+L_4+L_5 , a_f_3, b_f_3)

In [ ]:
## Modification des paramètres dans ANSYS Fluent

for i in range(len(Inputs)):
    named_expression = Inputs.iloc[i]['named_expression']
    value = Inputs.iloc[i]['value']
    unit = Inputs.iloc[i]['unit']
    jg.change_named_expression(tui, named_expression, value, unit)

In [ ]:
## Lancement de la simulation

solver.solution.run_calculation.iterate(number_of_iterations=1)

In [ ]:
## Récupération des flux de transfert thermique

df = jg.parse_report_to_dataframe(os.path.join(r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\gurvan_test','test.txt'),'heat_transfer')
# df = jg.parse_report_to_dataframe(os.path.join(r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\Reports','Report_heat_transfer.txt'),'heat_transfer')

hx_list =  ['hx_bend_mid_air' , 'hx_bend_mid_pv' , 'hx_bend_yd_air' , 'hx_bend_yd_pv' , 'hx_bend_yu_air' ,
    'hx_bend_yu_pv' , 'hx_flat_yd_air' , 'hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps' , 'hx_flat_yu_air' , 
    'hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps' , 'hx_sides_1' , 'hx_sides_2' , 'hx_sides_3' , 'hx_sides_4']

# exporter que radiativ + total pour choper le convectif

heat_transfer_hx_df = df[df['Component'].isin(hx_list)]
heat_transfer_hx_df

In [124]:
## Envoi des flux au PVT-thermal-performance-model

### Fin de la boucle